In [1]:
#pip install Dijkstar
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from dijkstar import Graph, find_path

E:\Anaconda\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [62]:
# create adjacency matrix
feature_df = pd.read_csv('AK_features.csv', header=None)
zero_matrix = np.zeros((len(feature_df), len(feature_df)))
adj_df = pd.DataFrame(zero_matrix)
id_map = {}

for i, id in enumerate(feature_df[0]):
    id_map[id] = i
    
id_map

{2050: 0,
 2164: 1,
 2060: 2,
 2070: 3,
 2170: 4,
 2013: 5,
 2158: 6,
 2185: 7,
 2180: 8,
 2016: 9,
 2150: 10,
 2282: 11,
 2066: 12,
 2063: 13,
 2230: 14,
 2105: 15,
 2130: 16,
 2188: 17,
 2220: 18,
 2290: 19,
 2020: 20,
 2198: 21,
 2195: 22,
 2275: 23,
 2110: 24,
 2122: 25,
 2068: 26,
 2240: 27,
 2090: 28,
 2100: 29}

In [77]:
pop_df = pd.read_csv('AK_pop.csv')

#Clean pop_df
pop_df = pop_df.rename(columns={"total pop = 733391": "pop"})
#pop_df['pop'] = pop_df['pop'].str.replace(' ', '').astype(int)
for i in range(len(pop_df)):
    pop_df.loc[i, 'pop'] = pop_df.loc[i,'pop'].split(' ')[1]
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df

,pop
0,18666
1,1476
2,844
3,4857
4,107081
5,3420
6,8368
7,11031
8,10046
9,5232


In [4]:
feature_df

,0,1,2
0,2050,18666,"02290,02170,02122,02158,02070,02164"
1,2164,1476,"02013,02060,02150,02122,02070,02050"
2,2060,844,"02070,02164"
3,2070,4857,"02060,02164,02050"
4,2170,107081,"02068,02290,02240,02063,02066,02020,02122,02050"
5,2013,3420,"02016,02164"
6,2158,8368,"02290,02180,02050"
7,2185,11031,"02290,02188"
8,2180,10046,"02290,02158,02188"
9,2016,5232,02013


In [5]:
for node, adj_list in zip(feature_df[0], feature_df[2]):
    node_idx = id_map[node]
    for adj in adj_list.split(','):
        adj_idx = id_map[int(adj[1:])]
        adj_df.iloc[node_idx, adj_idx] = 1
# adj_df
adj_matrix = adj_df.values

In [6]:
adj_matrix

array([[0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0

In [27]:
#Uses Dijkstra's algorithm to find the shortest distance between 2 points on the graph
graph = Graph()
for i in range(len(adj_matrix)):
    for j in range(len(adj_matrix[0])):
        if adj_matrix[i][j] == 1:
            graph.add_edge(i, j, 1)
find_path(graph, 1, 4)

PathInfo(nodes=[1, 0, 4], edges=[1, 1], costs=[1, 1], total_cost=2)

In [34]:
point1 = 2
point2 = 8
print("The shortest distance between points {} and {} is {}".format(point1, 
                                                           point2, 
                                                           find_path(graph, point1, point2)[3]))

The shortest distance between points 2 and 8 is 4


In [101]:
def normalize(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

normalize([0,3,6], 0, 1)

[0.0, 0.5, 1.0]

In [93]:
def evaluate(distribution, p_bar):

    district_dic = distribution[0]
    
    # calculate RMSE
    RMSE = 0
    #Add the the squared difference for every district to RMSE
    for i in district_dic:
        district_pop = 0
        district = district_dic[i]
        for j in district:
            district_pop += pop_df.loc[j, 'pop']
        RMSE += (district_pop - p_bar)**2
    #Divide RMSE by the number of districts, then take its square root to obtain the final RMSE
    RMSE = (RMSE / len(district_dic))**0.5
    print('RMSE = {}'.format(RMSE))
    

    # calculate compactness score
    CS = 0
    for k in district_dic:
        for i in district_dic[k]:
            for j in district_dic[k]:
                if i != j:
                    #Add shortest possible distance between vertices i and j
                    CS += find_path(graph, i, j)[3]
    print('CS = {}'.format(CS))

    return RMSE+CS # misschien normalizen zodat het tussen de 0 en 1 valt voor interpreteerbaarheid?

In [112]:
#Tests the evaluation function on a random distribution
#Change the value of k to change number of districts
k=4

distribution = create_random_distribution(4, 10)
print("Evaluating the following distribution:")
print(distribution)
p_bar = pop_df['pop'].sum() / k
print("Score = {}".format(evaluate(distribution, p_bar)))

Evaluating the following distribution:
({1: [3, 0, 1, 5, 6, 8, 9], 2: [2], 3: [7], 4: [4]}, 10)
RMSE = 146673.49683715016
CS = 94
Score = 146767.49683715016


In [90]:
distribution

({1: [8], 2: [2], 3: [0, 1, 3, 4, 5, 6, 7], 4: [9]}, 10)

In [10]:
def create_random_distribution(num_districts, num_units):
    
    keys = [i+1 for i in range(num_districts)]
    distribution = {key: [] for key in keys}
    assigned_units = []

    # randomly initialize district centers, one unit per district
    list_units = list(range(num_units))
    list_districts = list(range(1, num_districts+1))

    for d in range(num_districts):
        random_unit = random.choice(list_units)
        random_district = random.choice(list_districts)
        distribution[random_district].append(random_unit)
        list_units.remove(random_unit)
        list_districts.remove(random_district)
        assigned_units.append(random_unit)
        
    # add other units to districts based on adjacency matrix
    for unit in list_units:

        for i in range(num_units):

            if i in assigned_units and adj_matrix[unit, i] == 1:

                # find key of the unit
                for key, value_list in distribution.items():
                    if i in value_list:
                        found_key = key
                        break
                
                # append the unit to the district where there is also an adjacent unit
                distribution[found_key].append(unit)
                assigned_units.append(unit)
                break

        # in case a unit entered the loop but none of its adjacent units were assigned yet, it is randomly assigned to a district here
        if unit not in assigned_units:
            distribution[np.random.randint(1, num_districts+1)].append(unit)

    len_total = sum(len(lst) for lst in distribution.values())

    return distribution, len_total

In [84]:
create_random_distribution(4, 10)

({1: [3, 0, 1, 5, 6, 8, 9], 2: [4], 3: [2], 4: [7]}, 10)

In [12]:
# basic framework for simulated annealing step
def SimulatedAnnealing(objective, bounds, n_iterations, step_size, temp, num_districts, num_units):

    # generate initial district distribution
    best_solution = create_random_distribution(num_districts, num_units)

    # evaluate this distribution using evaluate
    best_evaluate = evaluate(best_solution)

    # current working solution
    current_distribution, current_evaluate = best_solution, best_evaluate
    
    for i in range(n_iterations):
        
        # take a step by randomly perturbing previous distribution
        candidate_distribution = perturb()

        # evaluate step using MSE
        candidate_evaluate = evaluate()
    
        # check for new best solution
        if candidate_evaluate < current_evaluate: # for minimization, use > for maximiation problem
            best_solution, best_evaluate = candidate_distribution, candidate_evaluate
            #print('>%d f(%s) = %.5f' % (i, best, best_eval))
    
        # calculate difference in scores
        diff = candidate_evaluate-current_evaluate
    
        # calculate temp for each epoch
        t = temp / float(i + 1)
    
        # calculate metropolis acceptance criterion
        metropolis = math.exp(-diff / t)
    
        # check whether we should keep new point
        if diff < 0 or np.random.rand() < metropolis:
            curr, curr_eval = candidate_distribution, candidate_evaluate

    return [best, best_eval]